In [1]:
import numpy as np
from scipy import integrate
from scipy.integrate import dblquad
from matplotlib import pyplot
import time

# Fem_1D_Quad

In [ ]:
def fem1d_quad_Neumann(N, interval, c_func, f_func, bc, der_order):
    lower, upper = interval
    h = (upper - lower) / N

    def ref_fi(x, i):
        # reference function
        if x < 0.0 or x > 1.0:
            return 0.0
        if i == 0:
            return 2.0 * x**2 - 3.0 * x + 1.0
        if i == 1:
            return 2.0 * x**2 - x
        if i == 2:
            return -4.0 * x**2 + 4.0 * x
    def ref_fi_1(x, i):
        if i == 0:
            return 4.0 * x - 3.0
        if i == 1:
            return 4.0 * x - 1.0
        if i == 2:
            return -8.0 * x + 4.0

    def fi(x, i, xn):
        # local function
        return ref_fi( (x - xn) / h, i)
    def fi_1(x, i, xn):
        # chain derivation
        return ref_fi_1( (x - xn) / h , i) / h

    def A_func(x, alpha, beta, xn):
        # integrand for computing system matrix entries
        return c_func(x) * fi_1(x, alpha, xn) * fi_1(x, beta, xn)

    def B_func(x, beta, xn):
        # integrand for computing right-hand side entries
        return f_func(x) * fi(x, beta, xn)
    
    # create an array of mesh nodes
    P = np.array([lower + n * h for n in range(0, N + 1)])
    # create an array of nodes' index in per mesh  
    T = np.array([range(0, N), range(1, N + 1)])

    # set up the finite element nodes
    Nb = 2 * N + 1
    Pb = np.array([lower + k * h / 2 for k in range(0, Nb)])
    Tb = np.array([range(0, Nb - 2, 2), range(2, Nb, 2), range(1, Nb - 1, 2)])

    # initialize the system matrix
    A = np.zeros(shape=(Nb, Nb))
    # loop over all mesh elements
    for n in range(0, N):
        # get endpoints of the current mesh element
        x_b = P[T[0, n]]
        x_u = P[T[1, n]]
        # loop over all hat functions for current element
        for alpha in range(0, 3):
            for beta in range(0, 3):
                # integrate A_func over current element to compute system matrix entries
                r, _ = integrate.quad(A_func, x_b, x_u, args = (alpha, beta, x_b))
                A[Tb[beta, n], Tb[alpha, n]] += r

    if der_order == 0:
        A[-1, :] = 0.0
        A[-1, -1] = 1.0
    else:
        A[0, :] = 0.0
        A[0, 0] = 1.0

    direction = [-1.0, 1.0]
    # initialize the right-hand side
    B = np.zeros(shape=(Nb, 1))
    # loop over all mesh elements
    for n in range(0, N):
        # get endpoints of the current mesh element
        x_b = P[T[0,n]]
        x_u = P[T[1,n]]
        # loop over all hat functions for current element
        for beta in range(0, 3):
            # integrate B_func over current element to compute right-hand side entries
            r, _ = integrate.quad(B_func, x_b, x_u, args=(beta, x_b))
            B[Tb[beta, n], 0] += r
    B[:,0] += direction[der_order] * bc[der_order]*c_func(interval[der_order])
    # apply boundary conditions to right-hand side
    if der_order == 0:
        B[-1, 0] = bc[1]
    else:
        B[0, 0] = bc[0]

    # solve the linear system to obtain the solution coefficients
    coefficients = np.linalg.solve(A, B)

    # return solution coefficients, grid points, system matrix, and right-hand side
    return coefficients, Pb, A, B


In [ ]:
def fem1d_quad_Dirichlet(N, interval, c_func, f_func, bc):
    # unpack interval into variables b and u
    b, u = interval
    # compute size of each subinterval
    h = (u - b) / N

    def ref_fi(x, i):
        # reference function
        if x < 0.0 or x > 1.0:
            return 0.0
        if i == 0:
            return 2.0 * x**2 - 3.0 * x + 1.0
        if i == 1:
            return 2.0 * x**2 - x
        if i == 2:
            return -4.0 * x**2 + 4.0 * x
    def ref_fi_1(x, i):
        if i == 0:
            return 4.0 * x - 3.0
        if i == 1:
            return 4.0 * x - 1.0
        if i == 2:
            return -8.0 * x + 4.0

    def fi(x, i, xn):
        # local function
        return ref_fi( (x - xn) / h, i)
    def fi_1(x, i, xn):
        # chain derivation
        return ref_fi_1( (x - xn) / h , i) / h

    def A_func(x, alpha, beta, xn):
        # integrand for computing system matrix entries
        return c_func(x) * fi_1(x, alpha, xn) * fi_1(x, beta, xn)

    def B_func(x, beta, xn):
        # integrand for computing right-hand side entries
        return f_func(x) * fi(x, beta, xn)

    # create an array of mesh nodes
    P = np.array([b + n * h for n in range(0, N + 1)])
    # create an array of nodes' index in per mesh  
    T = np.array([range(0, N), range(1, N + 1)])

    # set up the finite element nodes
    Nb = 2 * N + 1
    Pb = np.array([b + k * h / 2 for k in range(0, Nb)])
    Tb = np.array([range(0, Nb - 2, 2), range(2, Nb, 2), range(1, Nb - 1, 2)])

    # initialize the system matrix
    A = np.zeros(shape=(Nb, Nb))
    # loop over all mesh elements
    for n in range(0, N):
        # get endpoints of the current mesh element
        x_b = P[T[0, n]]
        x_u = P[T[1, n]]
        # loop over all hat functions for current element
        for alpha in range(0, 3):
            for beta in range(0, 3):
                # integrate A_func over current element to compute system matrix entries
                r, _ = integrate.quad(A_func, x_b, x_u, args = (alpha, beta, x_b))
                A[Tb[beta, n], Tb[alpha, n]] += r

    # apply Dirichlet boundary conditions to system matrix
    A[0, :] = 0.0
    A[0, 0] = 1.0
    A[-1, :] = 0.0
    A[-1, -1] = 1.0
    
    # initialize the right-hand side
    B = np.zeros(shape=(Nb, 1))
    # loop over all mesh elements
    for n in range(0, N):
        # get endpoints of the current mesh element
        x_b = Pb[Tb[0,n]]
        x_u = Pb[Tb[1,n]]
        # loop over all hat functions for current element
        for beta in range(0, 3):
            # integrate B_func over current element to compute right-hand side entries
            r, _ = integrate.quad(B_func, x_b, x_u, args=(beta, x_b))
            B[Tb[beta, n], 0] += r

    # apply boundary conditions to right-hand side
    B[0, 0] = bc[0]
    B[-1, 0] = bc[1]

    # solve the linear system to obtain the solution coefficients
    coefficients = np.linalg.solve(A, B)

    # return solution coefficients, grid points, system matrix, and right-hand side
    return coefficients, Pb, A, B



In [ ]:
def c_func(x):
    return np.exp(x)
def f_func(x):
    return -np.exp(x) * (np.cos(x) - 2 * np.sin(x) - x * np.cos(x) - x * np.sin(x))

In [ ]:

for i in range(1, 4):
    n = 2**i
    interval = [0.0, 1.0];
    bc = [0.0, np.cos(1.0)]

    startTime = time.time()
    u, x, A, B= fem1d_quad_Dirichlet(n, interval, c_func, f_func, bc)
    endTime = time.time()

    u = u.squeeze()
    u_real = x * np.cos(x)
    pyplot.plot(x, u, '.')
    pyplot.plot(x, u_real)
    print(f"N: {n}, Time: {endTime - startTime :.5f}s, Error: {np.max( np.abs(u - u_real) )}" )

---